In [5]:
# Cài đặt các thư viện cần thiết
!pip install yfinance pandas matplotlib plotly


In [6]:
# Import các thư viện
import os
import time
import random
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from google.colab import files

In [7]:
# 1. Thiết lập các tham số
STOCK_SYMBOLS = ['AAPL', 'GOOG', 'META']
INTERVAL = '1d'  # Thay đổi từ '1m' sang '1d' để giảm khả năng bị rate limit
DATA_DIR = '/content/stock_data'  # Thư mục lưu trữ dữ liệu trên Colab

# Tạo thư mục để lưu dữ liệu
!mkdir -p {DATA_DIR}

In [8]:
# 2. Hàm để lấy dữ liệu chứng khoán từ Yahoo Finance với cơ chế retry và delay
def fetch_stock_data(symbols, interval='1d', period='60d', max_retries=3):
    """Lấy dữ liệu chứng khoán từ Yahoo Finance với cơ chế retry và delay"""
    all_data = []

    for symbol in symbols:
        retries = 0
        success = False

        while retries < max_retries and not success:
            try:
                # Thêm delay ngẫu nhiên để tránh rate limit
                delay = 1 + random.random() * 2  # Delay từ 1-3 giây
                time.sleep(delay)

                print(f"Đang lấy dữ liệu cho {symbol} (lần thử {retries+1})...")

                # Lấy dữ liệu từ yfinance
                stock = yf.Ticker(symbol)
                data = stock.history(period=period, interval=interval)

                if data.empty:
                    print(f"Không có dữ liệu cho {symbol}, thử lại...")
                    retries += 1
                    continue

                data = data.reset_index()
                data['symbol'] = symbol

                # Chuyển đổi dữ liệu
                selected_data = data[['symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
                selected_data.columns = ['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume']

                # Đảm bảo các cột số được chuyển đổi thành float
                for col_name in ['open', 'high', 'low', 'close', 'volume']:
                    selected_data[col_name] = selected_data[col_name].astype(float)

                all_data.append(selected_data)
                print(f"✓ Đã lấy {len(selected_data)} dòng dữ liệu cho {symbol}")
                success = True

            except Exception as e:
                print(f"Lỗi khi lấy dữ liệu cho {symbol}: {e}")
                retries += 1
                delay = 3 + random.random() * 5  # Tăng delay lên 3-8 giây khi gặp lỗi
                print(f"Đợi {delay:.1f} giây trước khi thử lại...")
                time.sleep(delay)

        if not success:
            print(f"⚠️ Không thể lấy dữ liệu cho {symbol} sau {max_retries} lần thử!")

    return pd.concat(all_data) if all_data else pd.DataFrame()

In [9]:
# 3. Sử dụng dữ liệu mẫu nếu không thể lấy dữ liệu từ API
def generate_sample_data(symbols):
    """Tạo dữ liệu mẫu nếu không thể lấy từ API"""
    print("Tạo dữ liệu mẫu cho demo...")

    now = datetime.now()
    all_data = []

    for symbol in symbols:
        # Tạo 30 ngày dữ liệu
        dates = [now - timedelta(days=i) for i in range(30)]

        # Giá ban đầu khác nhau cho mỗi mã
        if symbol == 'AAPL':
            base_price = 150
        elif symbol == 'GOOG':
            base_price = 2500
        else:  # META
            base_price = 300

        data = []
        for i, date in enumerate(dates):
            # Tạo biến động giá hợp lý
            price_change = (random.random() - 0.5) * 10
            open_price = base_price + (i * 0.5) + (random.random() - 0.5) * 5
            close_price = open_price + price_change
            high_price = max(open_price, close_price) + random.random() * 3
            low_price = min(open_price, close_price) - random.random() * 3
            volume = int(random.random() * 10000000) + 5000000

            data.append({
                'symbol': symbol,
                'timestamp': date,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'volume': volume
            })

        all_data.append(pd.DataFrame(data))

    combined_data = pd.concat(all_data)
    combined_data = combined_data.sort_values(['symbol', 'timestamp'])

    return combined_data

In [10]:
# 4. Lưu dữ liệu vào file local
def save_to_local(df, folder_path):
    """Lưu DataFrame vào thư mục local trên Google Colab"""
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    file_path = f"{folder_path}/stock_data_{timestamp}.csv"

    df.to_csv(file_path, index=False)
    print(f"Đã lưu dữ liệu vào {file_path}")
    return file_path


In [11]:
# 5. Phân tích dữ liệu cơ bản
def analyze_data(dataframe):
    """Phân tích dữ liệu chứng khoán cơ bản"""
    results = {}

    # Tính các thống kê cơ bản theo cổ phiếu
    stats = dataframe.groupby('symbol').agg({
        'close': ['mean', 'min', 'max'],
        'volume': 'sum'
    })
    stats.columns = ['avg_price', 'min_price', 'max_price', 'total_volume']
    stats = stats.reset_index()

    # Tính biến động giá
    volatility = dataframe.copy()
    volatility['price_change'] = volatility['close'] - volatility['open']
    volatility['percent_change'] = (volatility['close'] - volatility['open']) / volatility['open'] * 100

    # Tính Moving Average
    moving_avg = []
    for symbol in dataframe['symbol'].unique():
        symbol_data = dataframe[dataframe['symbol'] == symbol].sort_values('timestamp').copy()
        if len(symbol_data) >= 10:
            symbol_data['moving_avg_5'] = symbol_data['close'].rolling(window=5).mean()
            symbol_data['moving_avg_10'] = symbol_data['close'].rolling(window=10).mean()
            moving_avg.append(symbol_data)

    moving_avg_df = pd.concat(moving_avg) if moving_avg else pd.DataFrame()

    results['stats'] = stats
    results['volatility'] = volatility
    results['moving_avg'] = moving_avg_df

    return results

In [13]:
# 6. Thêm các chỉ báo kỹ thuật
def add_technical_indicators(dataframe):
    """Thêm các chỉ báo kỹ thuật vào dữ liệu"""
    result = {}

    for symbol in dataframe['symbol'].unique():
        symbol_data = dataframe[dataframe['symbol'] == symbol].sort_values('timestamp').copy()

        # RSI - Relative Strength Index
        def calculate_rsi(data, window=14):
            delta = data.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
            # Xử lý trường hợp chia cho 0
            rs = gain / loss.replace(0, np.nan).fillna(gain)
            return 100 - (100 / (1 + rs))

        if len(symbol_data) >= 14:
            symbol_data['rsi'] = calculate_rsi(symbol_data['close'])

        # MACD (Moving Average Convergence Divergence)
        if len(symbol_data) >= 26:
            symbol_data['ema12'] = symbol_data['close'].ewm(span=12, adjust=False).mean()
            symbol_data['ema26'] = symbol_data['close'].ewm(span=26, adjust=False).mean()
            symbol_data['macd'] = symbol_data['ema12'] - symbol_data['ema26']
            symbol_data['signal'] = symbol_data['macd'].ewm(span=9, adjust=False).mean()
            symbol_data['macd_histogram'] = symbol_data['macd'] - symbol_data['signal']

        # Bollinger Bands
        window = 20
        if len(symbol_data) >= window:
            symbol_data['sma'] = symbol_data['close'].rolling(window=window).mean()
            symbol_data['std'] = symbol_data['close'].rolling(window=window).std()
            symbol_data['bollinger_upper'] = symbol_data['sma'] + (symbol_data['std'] * 2)
            symbol_data['bollinger_lower'] = symbol_data['sma'] - (symbol_data['std'] * 2)

        result[symbol] = symbol_data

    return result

In [14]:

# 7. Visualize dữ liệu
def visualize_data(stats_df, volatility_df, moving_avg_df):
    """Tạo biểu đồ trực quan từ dữ liệu phân tích"""
    # 7.1 Biểu đồ thanh hiển thị giá trung bình
    fig1 = px.bar(stats_df, x='symbol', y='avg_price', title='Giá trung bình theo cổ phiếu')
    fig1.show()

    # 7.2 Biểu đồ đường hiển thị biến động giá theo thời gian
    fig2 = px.line(volatility_df, x='timestamp', y='close', color='symbol',
                  title='Biến động giá cổ phiếu theo thời gian')
    fig2.show()

    # 7.3 Biểu đồ candlestick cho mỗi cổ phiếu
    for symbol in volatility_df['symbol'].unique():
        symbol_data = volatility_df[volatility_df['symbol'] == symbol]
        fig3 = go.Figure(data=[go.Candlestick(
            x=symbol_data['timestamp'],
            open=symbol_data['open'],
            high=symbol_data['high'],
            low=symbol_data['low'],
            close=symbol_data['close'],
            name=symbol
        )])
        fig3.update_layout(
            title=f'Biểu đồ Candlestick cho {symbol}',
            xaxis_title='Thời gian',
            yaxis_title='Giá',
            xaxis_rangeslider_visible=False
        )
        fig3.show()

    # 7.4 Biểu đồ đường cho Moving Average
    if not moving_avg_df.empty:
        for symbol in moving_avg_df['symbol'].unique():
            symbol_data = moving_avg_df[moving_avg_df['symbol'] == symbol].sort_values('timestamp')

            if 'moving_avg_5' not in symbol_data.columns:
                continue

            fig4 = go.Figure()
            fig4.add_trace(go.Scatter(x=symbol_data['timestamp'], y=symbol_data['close'],
                                    mode='lines', name='Close Price'))
            fig4.add_trace(go.Scatter(x=symbol_data['timestamp'], y=symbol_data['moving_avg_5'],
                                    mode='lines', name='MA 5', line=dict(dash='dash')))
            fig4.add_trace(go.Scatter(x=symbol_data['timestamp'], y=symbol_data['moving_avg_10'],
                                    mode='lines', name='MA 10', line=dict(dash='dot')))

            fig4.update_layout(
                title=f'Giá đóng cửa và Moving Average cho {symbol}',
                xaxis_title='Thời gian',
                yaxis_title='Giá',
                legend_title='Chỉ số'
            )
            fig4.show()

    # 7.5 Biểu đồ phần trăm thay đổi giá
    fig5 = px.box(volatility_df, x='symbol', y='percent_change',
                 title='Phân phối phần trăm thay đổi giá theo cổ phiếu')
    fig5.show()

In [15]:


# 8. Chạy toàn bộ pipeline
def run_stock_data_pipeline(save_files=False, use_sample_data=False):
    """Chạy toàn bộ pipeline xử lý dữ liệu chứng khoán"""
    try:
        # Tạo folder output
        output_folder = DATA_DIR

        # Lấy dữ liệu từ API hoặc tạo dữ liệu mẫu
        if not use_sample_data:
            print("Đang lấy dữ liệu chứng khoán từ API...")
            stock_data = fetch_stock_data(STOCK_SYMBOLS, INTERVAL, period='60d', max_retries=3)

            if stock_data.empty:
                print("Không thể lấy dữ liệu từ API! Chuyển sang sử dụng dữ liệu mẫu...")
                stock_data = generate_sample_data(STOCK_SYMBOLS)
        else:
            print("Sử dụng dữ liệu mẫu theo yêu cầu...")
            stock_data = generate_sample_data(STOCK_SYMBOLS)

        # Hiển thị dữ liệu gốc
        print("\n--- DỮ LIỆU CHỨNG KHOÁN ---")
        print(stock_data.head(10))
        print(f"Tổng số dòng dữ liệu: {len(stock_data)}")

        # Phân tích dữ liệu
        print("\nĐang phân tích dữ liệu...")
        analysis_results = analyze_data(stock_data)

        # Hiển thị kết quả phân tích
        print("\n--- KẾT QUẢ PHÂN TÍCH ---")
        print("\nThống kê giá trung bình:")
        print(analysis_results["stats"].to_string(index=False))

        # Hiển thị biến động giá
        print("\nBiến động giá (10 dòng đầu tiên):")
        volatility_sample = analysis_results["volatility"].sort_values('timestamp', ascending=False).head(10)
        print(volatility_sample[['symbol', 'timestamp', 'open', 'close', 'price_change', 'percent_change']].to_string(index=False))

        # Thêm chỉ báo kỹ thuật
        print("\nĐang thêm chỉ báo kỹ thuật...")
        tech_indicators = add_technical_indicators(stock_data)

        # Hiển thị chỉ báo kỹ thuật
        print("\nChỉ báo kỹ thuật (dòng cuối cùng):")
        for symbol, data in tech_indicators.items():
            if not data.empty:
                latest = data.iloc[-1]
                print(f"\n{symbol}:")
                if 'macd' in latest and not pd.isna(latest['macd']):
                    print(f"  MACD: {latest['macd']:.4f}, Signal: {latest['signal']:.4f}")
                if 'bollinger_upper' in latest and not pd.isna(latest['bollinger_upper']):
                    print(f"  Bollinger Bands: Upper = {latest['bollinger_upper']:.2f}, Lower = {latest['bollinger_lower']:.2f}")
                if 'rsi' in latest and not pd.isna(latest['rsi']):
                    print(f"  RSI: {latest['rsi']:.2f}")

        # Visualize dữ liệu
        print("\nĐang tạo biểu đồ...")
        visualize_data(
            analysis_results["stats"],
            analysis_results["volatility"],
            analysis_results["moving_avg"]
        )

        # Lưu dữ liệu nếu được yêu cầu
        if save_files:
            print("\nĐang lưu dữ liệu vào file local...")
            local_path = save_to_local(stock_data, output_folder)

            # Lưu kết quả phân tích
            analysis_results["stats"].to_csv(f"{output_folder}/stats_analysis.csv", index=False)
            analysis_results["volatility"].to_csv(f"{output_folder}/volatility_analysis.csv", index=False)

            # Tạo tệp zip để tải xuống
            !zip -r /content/stock_analysis.zip {output_folder}/*.csv

            # Tải xuống tệp zip
            files.download('/content/stock_analysis.zip')

            print("Đã lưu tất cả dữ liệu!")
        else:
            print("\nKhông lưu dữ liệu - Chỉ hiển thị kết quả phân tích.")

        print("\nPipeline hoàn thành!")

    except Exception as e:
        print(f"Lỗi trong pipeline: {e}")


In [2]:
# Xử lý dữ liệu chứng khoán với giải pháp cho vấn đề Rate Limit

# Chạy pipeline
if __name__ == "__main__":
    print("\n=== PHÂN TÍCH DỮ LIỆU CHỨNG KHOÁN ===")
    print("1. Thử lấy dữ liệu từ API")
    print("2. Sử dụng dữ liệu mẫu")
    choice = input("\nNhập lựa chọn của bạn (1-2): ")

    use_sample = (choice == "2")

    print("\nChạy pipeline phân tích dữ liệu chứng khoán...")
    run_stock_data_pipeline(save_files=False, use_sample_data=use_sample)


=== PHÂN TÍCH DỮ LIỆU CHỨNG KHOÁN ===
1. Thử lấy dữ liệu từ API
2. Sử dụng dữ liệu mẫu

Nhập lựa chọn của bạn (1-2): 2

Chạy pipeline phân tích dữ liệu chứng khoán...
Sử dụng dữ liệu mẫu theo yêu cầu...
Tạo dữ liệu mẫu cho demo...

--- DỮ LIỆU CHỨNG KHOÁN ---
   symbol                  timestamp        open        high         low  \
29   AAPL 2025-01-30 05:59:09.650534  164.020324  165.628923  159.455246   
28   AAPL 2025-01-31 05:59:09.650534  162.066406  166.709545  161.187068   
27   AAPL 2025-02-01 05:59:09.650534  164.591624  166.484378  159.636192   
26   AAPL 2025-02-02 05:59:09.650534  164.573143  166.202615  160.511987   
25   AAPL 2025-02-03 05:59:09.650534  160.355203  162.730311  154.460132   
24   AAPL 2025-02-04 05:59:09.650534  163.138453  164.103310  160.366768   
23   AAPL 2025-02-05 05:59:09.650534  163.203220  166.475575  162.284848   
22   AAPL 2025-02-06 05:59:09.650534  160.756206  161.430473  159.737982   
21   AAPL 2025-02-07 05:59:09.650534  162.294932  165.5


Không lưu dữ liệu - Chỉ hiển thị kết quả phân tích.

Pipeline hoàn thành!


In [ ]:
run_stock_data_pipeline(save_files=True,
                        use_sample_data=use_sample)